In [56]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore


import pandas as pd


In [57]:
pd.set_option('display.max_columns', None)


In [58]:
# Inicializa o Firebase Admin, se ainda não estiver inicializado
if not firebase_admin._apps:
    cred = credentials.Certificate('firebase-key.json')
    firebase_admin.initialize_app(cred)

db = firestore.client()

In [59]:
def listar_colecoes_e_documentos(head = True):
    colecoes = db.collections()  # Lista todas as coleções
    for colecao in colecoes:
        print(f'Nome da Coleção: {colecao.id}')
        documentos = colecao.stream()
        for idx, documento in enumerate(documentos):
            print(f' - Documento ID: {documento.id}, Dados: {documento.to_dict()}')
            if (head == True) and (idx == 9):
                break

# Chamada da função
listar_colecoes_e_documentos()

Nome da Coleção: CNPJ
 - Documento ID: 03812120000181_false, Dados: {'CNPJ__c': '03.812.120/0001-81', 'Email': 'borracheiro@teste.com'}
 - Documento ID: 03974743000150_false, Dados: {'CNPJ__c': '03.974.743/0001-50', 'Email': 'tester@teste.com'}
 - Documento ID: 04522171000131_false, Dados: {'CNPJ__c': '04.522.171/0001-31', 'Email': 'caroool@carol.com'}
 - Documento ID: 11934656000106_false, Dados: {'CNPJ__c': '11.934.656/0001-06', 'Email': 'teste1@teste.com'}
 - Documento ID: 16046110000124_false, Dados: {'CNPJ__c': '16.046.110/0001-24', 'Email': 'teste92@teste.com'}
 - Documento ID: 30449012000169_false, Dados: {'CNPJ__c': '30.449.012/0001-69', 'Email': 'n@teste.com'}
 - Documento ID: 35118924000171_false, Dados: {'CNPJ__c': '35.118.924/0001-71', 'Email': 'novoos@teste.com'}
 - Documento ID: 35895109000119_false, Dados: {'CNPJ__c': '35.895.109/0001-19', 'Email': 'nooo@teste.com'}
 - Documento ID: 37668182000183_false, Dados: {'CNPJ__c': '37.668.182/0001-83', 'Email': 'teste@jose.com'}

In [60]:
def colecao_para_dataframe(nome_colecao):
    colecao_ref = db.collection(nome_colecao)
    documentos = colecao_ref.stream()

    # Criar uma lista para armazenar os dados dos documentos
    documentos_lista = []

    # Iterar sobre cada documento e adicionar ao dicionário, em seguida, adicionar à lista
    for doc in documentos:
        doc_dict = doc.to_dict()
        doc_dict['document_id'] = doc.id  # Incluir o ID do documento, se necessário
        documentos_lista.append(doc_dict)

    # Converter a lista de dicionários em um DataFrame
    if documentos_lista:
        dataframe = pd.DataFrame(documentos_lista)
    else:
        dataframe = pd.DataFrame()  # Retorna um DataFrame vazio se não houver documentos

    return dataframe


In [61]:
def criar_documento(nome_colecao, dados, doc_id=None):
    """
    Cria um novo documento em uma coleção especificada no Firestore.

    :param nome_colecao: String que especifica o nome da coleção.
    :param dados: Dicionário contendo os dados a serem inseridos no novo documento.
    :param doc_id: String opcional que especifica o ID do documento. Se não for fornecido,
                   o Firestore gera um ID único automaticamente.
    """
    colecao_ref = db.collection(nome_colecao)

    if doc_id:
        doc_ref = colecao_ref.document(doc_id)
        doc_ref.set(dados)
        print(f'Documento criado com ID: {doc_id}')
    else:
        doc_ref = colecao_ref.add(dados)
        print(f'Documento criado com ID gerado pelo Firestore: {doc_ref[1].id}')
        
    return str(doc_ref[1].id)

def ler_todos_documentos(nome_colecao):
    """
    Lê e imprime todos os documentos de uma coleção especificada no Firestore.

    :param nome_colecao: String que especifica o nome da coleção a ser lida.
    """
    colecao_ref = db.collection(nome_colecao)
    documentos = colecao_ref.stream()
    
    for doc in documentos:
        print(f'{doc.id} => {doc.to_dict()}')

def ler_documento(nome_colecao, doc_id):
    """
    Lê e imprime um documento específico de uma coleção pelo seu ID.

    :param nome_colecao: String que especifica o nome da coleção.
    :param doc_id: String que especifica o ID do documento a ser lido.
    """
    doc_ref = db.collection(nome_colecao).document(doc_id)
    doc = doc_ref.get()
    if doc.exists:
        print(f'Documento encontrado: {doc.id} => {doc.to_dict()}')
    else:
        print('Documento não encontrado')

def atualizar_documento(nome_colecao, doc_id, novos_dados):
    """
    Atualiza um documento específico em uma coleção com novos dados.

    :param nome_colecao: String que especifica o nome da coleção.
    :param doc_id: String que especifica o ID do documento a ser atualizado.
    :param novos_dados: Dicionário contendo os novos dados para atualizar o documento.
    """
    doc_ref = db.collection(nome_colecao).document(doc_id)
    doc_ref.update(novos_dados)
    print(f'Documento {doc_id} atualizado com sucesso.')

def deletar_documento(nome_colecao, doc_id):
    """
    Deleta um documento específico de uma coleção pelo seu ID.

    :param nome_colecao: String que especifica o nome da coleção.
    :param doc_id: String que especifica o ID do documento a ser deletado.
    """
    db.collection(nome_colecao).document(doc_id).delete()
    print(f'Documento {doc_id} deletado com sucesso.')



___

### Exemplo

In [62]:
id_criado = criar_documento('CPF', {'CPF__c': '00735256454', 'Email': 'caleo@emailcom'})
# Exemplo de uso
nome_colecao = 'CPF'
df = colecao_para_dataframe(nome_colecao)
display(df[df['document_id'] == id_criado])

Documento criado com ID gerado pelo Firestore: vt8pvsgRNCm772UGYwM3


,CPF__c,Email,document_id
83,00735256454,caleo@emailcom,vt8pvsgRNCm772UGYwM3


In [63]:
id_list = df[df['Email'] == 'caleo@emailcom']['document_id'].values

In [64]:
for id_doc in id_list:
  deletar_documento('CPF', id_doc)
  
nome_colecao = 'CPF'
df = colecao_para_dataframe(nome_colecao)
display(df)

Documento vt8pvsgRNCm772UGYwM3 deletado com sucesso.


,CPF__c,Email,document_id
0,001.553.690-40,josecleiton@jose.com,00155369040_false
1,023.972.440-20,h@teste.com,02397244020_true
2,029.661.230-87,teste1@teste.com,02966123087_false
3,032.938.990-48,testwork@jose.jose,03293899048_false
4,046.408.520-96,caroooooo@b.com,04640852096_false
...,...,...,...
78,883.488.070-63,teste98@teste.com,88348807063_true
79,893.469.240-55,keusantos@jose.com,89346924055_true
80,930.360.100-92,g@gmail.com,93036010092_false
81,986.000.590-70,novoteste@teste.com.br,98600059070_false
